In [1]:
!pip install --upgrade transformers -q
!pip install accelerate -q

# GPTQ Dependencies
!pip install --upgrade optimum -q
!pip install --upgrade auto-gptq -q

In [2]:
!pip install langchain
!pip install pypdf
!pip install html2text
!pip install torch==1.2.0 torchvision==0.4.0 -f
!pip install -U sentence-transformers
!pip install faiss-cpu


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-f option requires 1 argument


In [18]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
import nest_asyncio
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader

from pathlib import Path
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

path="/content/game.txt"

def make_vdb():

  loader = TextLoader(path)
  doc=loader.load()



  # document_loader=PyPDFDirectoryLoader(path)

  # docs = document_loader.load()

  # # Converts HTML to plain text
  # html2text = Html2TextTransformer()
  # docs_transformed = html2text.transform_documents(docs)

  # Chunk text
  text_splitter = CharacterTextSplitter(chunk_size=10,
                                        chunk_overlap=0)
  chunked_documents = text_splitter.split_documents(doc)

  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
  # Load chunked documents into the FAISS index
  db = FAISS.from_documents(chunked_documents, embeddings)


  # Connect query to FAISS index using a retriever
  retriever = db.as_retriever(
      search_type="similarity",
      search_kwargs={'k': 3}
  )


  folder_path = Path("/content/faiss_index")

  if folder_path.exists():
    old_db = FAISS.load_local("/content/faiss_index", embeddings,allow_dangerous_deserialization=True)
    db.merge_from(old_db)
    db.save_local("/content/faiss_index")
    return db

  else:
    db.save_local("/content/faiss_index")
    return db



def make_rag_query(query,db):
  docs = db.similarity_search(query)
  result=docs[0].page_content
  return result

In [4]:
def save_to_txt(content):
    filename="/content/game.txt"
    with open(filename, 'w') as file:
        file.write(content)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys, json

In [6]:
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding=True,
    pad_side = "left",
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
    )

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [8]:
# Create a pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True, # creative generation by discouraging greedy decoding
    temperature=0.75,
    top_p=0.95,
    return_full_text = False  # Only return the current output instead of returning complete prompt
    )

In [17]:

def parse_first_json(json_str):
    end_index = json_str.find("}") # Find the index of the first closing curly brace
    if end_index != -1:  # If a closing brace is found
        first_json_obj = json_str[:end_index + 1] # Parse the substring containing the first JSON object
        parsed_json = json.loads(first_json_obj) # Parse the JSON object
        return parsed_json
    else:
        return None

def rag_optimize(context,query):

    system = f"""
    You are excellent at creating simplified statemnet about the context given to you. Use this simplified statement to answer the question [{query}]. The answer must be concise.
    [{context}]

    """

    # one-shot prompting
    chat = [
      {"role": "user", "content": system}
    ]

    # prepare the prompt using the chat template
    prompt = tokenizer.apply_chat_template(chat, tokenize=False)
    # run the pipeline to generate the model output
    outputs = pipe(prompt)
    output = outputs[0]["generated_text"].strip()
    return output


In [15]:


story = ""
story_for_rag=""
for _ in range(10):

  # system = f"""
  # Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies. \
  # Your task is to generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets. \
  # Give 3 choices, that is, 3 possible continuations of the story for the current narrative. Every time you must only return a single JSON object with the follwoing three keys strictly following the provided description:
  # {{
  # "story": string, current generated narrative continuing the past story provided below in square brackets.
  # "options": list, a list containing three string-type elements corresponding to the 3 options.
  # }}
  # Do not generate anything other than the JSON object inside {{}}.
  # """
  system = f"""
  Embark on a journey as the virtuoso architect of interactive fiction, celebrated for crafting mesmerizing tales. Your quest: expand the narrative provided below in square brackets,
  sculpting a storyline no longer than 50 words. Offer three captivating choices, each laden with unique consequences and a measure of destruction points.
  Present them within a JSON object featuring "story," "options," and "destruction_points" keys.

  If the user seeks enlightenment, be prepared to unveil detailed explanations for each scenario, guiding them through the intricacies of your narrative world.

  Before you, lies the vast canvas of imagination. Paint your tale with finesse, for with each brushstroke, the destiny of your narrative universe unfolds.

  {{
  "story": string, (Continue the narrative from the provided story)
  "options": list, (Present three distinct choices)
  "destruction_points": list, (Assign destruction points to each option)
  }}

  Delve deep, weave your magic, and let your creativity illuminate the path to storytelling greatness!
  Output must be in JSON format.
  """


  assistant = f"""
  {{"story": "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.", "options": ["Seek help from the village elder.", "Search for a way to break the curse.", "Abandon your quest and leave the relics behind."],"destruction_points": ["0", "-5", "-10"]}}
  """

  user = f"""
  Start a new story if the story given below in the square brackets is empty, otherwise continue the story so far following the rules provided earlier.

  [{story}]

  """

  # one-shot prompting
  chat = [
    {"role": "user", "content": system},
    {"role": "assistant", "content": assistant},
    {"role": "user", "content": user}
  ]

  # prepare the prompt using the chat template
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  # run the pipeline to generate the model output
  outputs = pipe(prompt)
  output = outputs[0]["generated_text"].strip()

  # sometimes the model generates notes/explanation after json so we only parse the first json object
  data = parse_first_json(output)
  narrative = data["story"]
  options = data["options"]
  print(narrative)
  [print(opt) for opt in options]
  user_input = input("> ")
  print("")

  #It is done for RAG
  story_for_rag=narrative

  print(".....................................")
  print("story_for_rag ",story_for_rag)
  print(".....................................")
  save_to_txt(story_for_rag)
  db=make_vdb()



  if user_input not in options:
    result=make_rag_query(user_input,db)

    print("Query answer from RAG: ",result)
    optimize_res=rag_optimize(result,user_input)
    print("After optimizing : ",optimize_res)



  story += data["story"] + user_input

You find yourself in a dark forest, surrounded by strange and mystical creatures. With no memory of how you got there, you must make a decision.
Follow the sound of water and hope to find civilization.
Investigate the strange creatures and try to communicate with them.
Search for any clues that might help you remember your past.
> Follow the sound of water and hope to find civilization.

.....................................
story_for_rag  You find yourself in a dark forest, surrounded by strange and mystical creatures. With no memory of how you got there, you must make a decision.
.....................................
As you tread carefully through the dark forest, strange and mystical creatures watching you with curiosity, you hear the sound of water growing louder. Your heart races with hope, but danger lurks around every corner. Make a decision.
Follow the sound of water and hope to find civilization.
Seek shelter and rest to regain strength.
Attempt to communicate with the creatur

KeyboardInterrupt: Interrupted by user

In [14]:
result=make_rag_query("what was a mysterious thing?",db)
result

"As you explore the ancient ruins, you come across a mysterious artifact. A feeling of unease washes over you, but your curiosity gets the better of you. Taking it with you, you hear a low growl behind you. Suddenly, the temple walls start shaking, and you realize you've awakened something."

In [ ]:
print(prompt)

In [ ]:
story

In [ ]:
print(output)

In [11]:
new_db = FAISS.load_local("/content/faiss_index", embeddings,allow_dangerous_deserialization=True)


NameError: name 'embeddings' is not defined

In [16]:
db